<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/header.png?raw=1" width="1000"/>


# Mapeo de inundaciones mediante imágenes de radar Sentinel-1

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/example.png?raw=1" width="1000"/>

***

El objetivo de esta [práctica recomendada](https://un-spider.org/advisory-support/recommended-practices) es determinar la extensión de las áreas inundadas. El uso de imágenes satelitales de radar de apertura sintética (SAR) para el mapeo de la extensión de las inundaciones constituye una solución viable con un procesamiento rápido de imágenes, que proporciona información de inundaciones casi en tiempo real a las agencias de ayuda para apoyar la acción humanitaria. La alta confiabilidad de los datos, así como la ausencia de restricciones geográficas, como la accesibilidad del sitio, enfatizan el potencial de esta tecnología.

Este cuaderno de Jupyter Notebook cubre toda la cadena de procesamiento desde la consulta de datos, la descarga, hasta la exportación de un producto final de máscara de inundación mediante la utilización de imágenes de acceso libre SAR Sentinel-1. El flujo de trabajo de la herramienta sigue la práctica recomendada de ONU-SPIDER sobre la [cartografía de inundaciones basada en radar](https://un-spider.org/advisory-support/recommended-practices/recommended-practice-radar-based-flood-mapping), tal y como se ilustra en el siguiente cuadro. Después de ingresar las especificaciones del usuario, los datos de Sentinel-1 se pueden descargar directamente desde el <a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a>. Posteriormente, los datos se procesan y almacenan en una variedad de formatos de salida.

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart0.png?raw=1" width="1000"/>

***

***Estructura del archivo***  
El archivo Jupyter Notebook constituye el directorio de origen. Los datos adicionales están contenidos en subcarpetas. Las imágenes de Sentinel-1 deben almacenarse en una subcarpeta llamada *'entrada'*. Si no se proporciona ninguna imagen, la subcarpeta se creará automáticamente al acceder y descargar datos a través de la herramienta<a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a>. Si hay un archivo de área de interés (AOI) disponible (formatos compatibles: GeoJSON, SHP, KML, KMZ), debe colocarse en una subcarpeta llamada *'AOI'*. Si no hay ninguno, un mapa interactivo permitirá dibujar manualmente el área de interés. Por razones de selección automática de archivos, se recomienda colocar solo un archivo AOI en la carpeta correspondiente. Sin embargo, si existen varios archivos, los archivos GeoJSON tendrán prioridad, seguidos de SHP, KML y KMZ. Los datos procesados se almacenan en una subcarpeta llamada * 'salida' *.  
Para ejecutar la herramienta sin interacción del usuario, todas las entradas deben estar claramente definidas. Esto significa que la subcarpeta *'input'* debe incluir una sola imagen Sentinel-1 y la subcarpeta *'AOI'* un solo archivo de área de interés AOI. Todos los demás escenarios requieren interacción manual, como descargar datos o definir un AOI. 

***Limitaciones***  
Existen limitaciones a la hora de detectar vegetación inundada e inundaciones en áreas urbanas debido a la retrodispersión de doble rebote. Si el agua y el no agua se distribuyen de manera muy desigual en la imagen, es posible que el histograma no tenga un mínimo local claro, lo que da lugar a resultados incorrectos en el proceso de binarización automática.


***Importante***
El cuaderno de Jupyter Notebook aprovecha la API <a href="https://step.esa.int/docs/v6.0/apidoc/engine/"> ESA SNAP </a> y requiere la instalación de la <i> ágil interfaz </i> SNAP-Python. Haga clic <a href="https://senbox.atlassian.net/wiki/spaces/SNAP/pages/24051781/Using+SNAP+in+your+Python+programs"> aquí </a> para obtener más información. Además, las <a href="https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html"> extensiones de Jupyter Notebook </a> *Codefolding*, *ExecuteTime* y *Table of Contents (2)* se utilizan para un mejor rendimiento.

***

## Entradas de Usuario

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart1.png?raw=1" width="1000"/>

Especifique en la celda de código a continuación **i)** el tipo de polarización que se procesará, **ii)** si los datos se descargarán del <a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a> con el respectivo período de detección y detalles de inicio de sesión, y **iii)** si los resultados intermedios deben ser graficados durante el proceso.

In [ ]:
# polarizaciones a procesar
polarisations = 'VH'                              # 'VH', 'VV', 'ambas'

# descargar imagen del Copernicus Open Access Hub
download = {
    'imageDownload'     : True,                   # 'True', 'False'
    'period_start'      : [2021, 1, 1],           # formato: [año, mes, día]
    'period_stop'       : [2021, 1, 8],           # formato: [año, mes, día]
    'username'          : 'username',             # nombre de ususario
    'password'          : 'password'              # contraseña
}

# mostar resultados intermedios si se selecciona 'True'
plotResoluts = True                               # 'True', 'False'

## Inicialización

Esta sección carga los módulos de Python relevantes para el siguiente análisis e inicializa las funcionalidades básicas.

In [ ]:
# Click para iniciar

#####################################################
###################### IMPORTAR #####################
#####################################################

# MODULO                                      # DESCRIPCIÓN
import sys
import matplotlib.pyplot as plt               # crear visualizaciones
import numpy as np                            # Computación científica
import json                                   # codificación y decodifiación JSON 
import glob                                   # acceso de datos
import os                                     # acceso de datoss
import ipywidgets                             # controles interactivos de la UI 
import time                                   # tiempo de evaluación 
import shutil                                 # operaciones de archivo
import ipyleaflet                             # visualización
import geopandas                              # manipulación y análisis de datos
import snappy                                 # interfase SNAP Python 
import jpy                                    # puente Python-Java 
import skimage.filters                        # cálculo de umbral
import functools                              # funciones y operaciones de orden superior
from ipyfilechooser import FileChooser        # widget selector de archivos
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt  # interface para el Open Access Hub
from datetime import date                     # fechas, horas e intervalos
from IPython.display import display           # visualización
from osgeo import ogr, gdal, osr              # conversión de datos
from zipfile import ZipFile                   # gestión de archivos




####################################################
############## DEFINICIÓN DE FUNSIONES #############
####################################################

# Esta función busca el archivo de AOI, se convierte a GeoJSON si no se proporciona uno y devuelve GeoJSON
def readJSONFromAOI(path):
    # busque el archivo GeoJSON en la subcarpeta 'AOI'
    if len(glob.glob('%s/*.geojson' % path)) == 1:
        file = glob.glob('%s/*.geojson' % path)[0]
    elif len(glob.glob('%s/*.json' % path)) == 1:
        file = glob.glob('%s/*.json' % path)[0]

    # convertir SHP a GeoJSON si no se proporciona un JSON
    elif len(glob.glob('%s/*.shp' % path)) == 1:
        file_name = os.path.splitext(glob.glob('%s/*.shp' % path)[0])[0].split('/')[-1]
        shp_file = geopandas.read_file(glob.glob('%s/*.shp' % path)[0])
        shp_file.to_file('%s/%s.json' % (path, file_name), driver='GeoJSON')
        file = glob.glob('%s/*.json' % path)[0]

    # convertir KML a GeoJSON si no se proporciona un JSON o SHP
    elif len(glob.glob('%s/*.kml' % path)) == 1:
        file_name = os.path.splitext(glob.glob('%s/*.kml' % path)[0])[0].split('/')[-1]
        kml_file = gdal.OpenEx(glob.glob('%s/*.kml' % path)[0])
        ds = gdal.VectorTranslate('%s/%s.json' % (path, file_name), kml_file, format='GeoJSON')
        del ds
        file = glob.glob('%s/*.json' % path)[0]

    # convertir KMZ a JSON si no se proporciona un JSON, SHP o KML
    elif len(glob.glob('%s/*.kmz' % path)) == 1:
        # abrir archivo KMZ y extraer los datos
        with ZipFile(glob.glob('%s/*.kmz' % path)[0], 'r') as kmz:
            folder = os.path.splitext(glob.glob('%s/*.kmz' % path)[0])[0]
            kmz.extractall(folder)
        # convertir KML a GeoJSON si la carpeta extraída contiene un archivo KML
        if len(glob.glob('%s/*.kml' % folder)) == 1:
            kml_file = gdal.OpenEx(glob.glob('%s/*.kml' % folder)[0])
            ds = gdal.VectorTranslate('%s/%s.json' % (path, folder.split('/')[-1]), kml_file, format='GeoJSON')
            del ds
            file = glob.glob('%s/*.json' % path)[0]
            # remove unzipped KMZ directory and data
            shutil.rmtree(folder)
    # eliminar el directorio y los datos del KMZ descomprimidos
    else:
        raise FileNotFoundError

    # abrir archivo JSON y almacenar los datos
    with open(file, 'r') as f:
        data_json = json.load(f)

    return data_json

# gráficar la banda e histograma de entrada y un umbral de tipo de 'Banda'
# SNAP API: https://step.esa.int/docs/v6.0/apidoc/engine/
def plotBand(band, threshold, binary=False):
    # realce de color
    vmin, vmax = 0, 1
    # leer los valores de pixeles
    w = band.getRasterWidth()
    h = band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h, w
    # realce de color
    if binary:
        cmap = plt.get_cmap('binary')
    else:
        vmin = np.percentile(band_data, 2.5)
        vmax = np.percentile(band_data, 97.5)
        cmap = plt.get_cmap('gray')
    # gráficar la banda
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    ax1.imshow(band_data, cmap=cmap, vmin=vmin, vmax=vmax)
    ax1.set_title(band.getName())
    # gráficar el histograma
    band_data.shape = h * w 
    ax2.hist(np.asarray(band_data[band_data != 0], dtype='float'), bins=2048)
    ax2.axvline(x=threshold, color='r')
    ax2.set_title('Histogram: %s' % band.getName())
    
    for ax in fig.get_axes():
        ax.label_outer()




####################################################
###################### CÓDIGO ######################
####################################################
        
# obtener el directorio de trabajo actual
directory = os.getcwd()

## Descarga de Imagen

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart2.png?raw=1" width="1000"/>

Esta sección permite el acceso y la descarga de datos interactivos desde el <a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a>. Si se proporciona un archivo AOI en la subcarpeta *'AOI'*, la herramienta buscará y muestrará las imágenes Sentinel-1 disponibles. Si no se proporciona un archivo AOI, la barra de búsqueda en el lado izquierdo del mapa interactivo se puede utilizar para encontrar la región deseada. A continuación, el archivo AOI se puede seleccionar y editar manualmente mediante la herramienta de dibujo. Al hacer clic en el botón Buscar debajo del mapa, se cargarán las imágenes disponibles. Si se extraen varias AOI, solo se considera la última. Al pasar el cursor sobre una imagen de Sentinel-1, se muestran el índice de mosaico y la fecha de ingestión. La siguiente tabla resume la información sobre todos los mosaicos disponibles y permite la descarga. Los datos se almacenan en la subcarpeta *'entrada'* creada automáticamente. Open Access Hub mantiene un archivo en línea de al menos el último año de productos para su descarga inmediata. El acceso a productos anteriores que ya no están disponibles en línea activará automáticamente la recuperación de los archivos a largo plazo. La descarga real puede ser iniciada por el usuario una vez que se restauran los datos (dentro de las 24 horas).

In [ ]:
# Click para iniciar

####################################################
############# DEFINICIÓN DE LA FUNCIÓN #############
####################################################

# buscar y mostrar los mosaicos de Sentinel-1 disponibles
def queri(footprint):
    # imprime el estado
    print('Loading...', flush=True)
    # buscar productos en el Copernicus Open Access Hub con respecto a la zona de entrada y el período de detección
    try:
        products = api.query(footprint,
                             date = (date(download['period_start'][0], download['period_start'][1], download['period_start'][2]),
                                     date(download['period_stop'][0], download['period_stop'][1], download['period_stop'][2])),
                             platformname = 'Sentinel-1',
                             producttype = 'GRD')
        print('Successfully connected to Copernicus Open Access Hub.\n', flush=True)
    except:
        sys.exit('\nLogin data not valid. Please change username and/or password.')
    # convierte a GeoJSON para graficar
    products_json = api.to_geojson(products)
    # generar una advertencia de que no hay imagen disponible en un período de detección dado
    if not products_json['features']:
        sys.exit('\nNo Sentinel-1 images available. Please change sensing period in user input section.')
    # convertir a un dataframe para visualización de tablas
    products_df = api.to_dataframe(products).sort_values('ingestiondate', ascending=[False])
    # adicionar un índice al dataframe
    indices = []
    for i in range (1, len(products_df.index)+1):
        indices.append('Tile %d' % i)
        products_json.features[i-1].properties['index'] = ' Tile %d' % i
    products_df.insert(0, 'index', indices, True) 
    # graficar los mosaicos de Sentinel-1 disponibles
    geo_json = ipyleaflet.GeoJSON(data = products_json,
                                  name = 'S1 tiles',
                                  style = {'color' : 'royalblue'},
                                  hover_style = {'fillOpacity' : 0.4})
    download_map.add_layer(geo_json)
    # llamar a la función click_handler cuando el usuario hace clic en el mosaico de Sentinel-1
    tile_ID = geo_json.on_hover(hover_handler)
    # imprimir la tabla con botones de descarga
    grid = ipywidgets.GridspecLayout(len(products_df.index)+1, 5, height='250px')
    grid[0,0] = ipywidgets.HTML('<h4>Index</h4>')
    grid[0,1] = ipywidgets.HTML('<h4>Ingestion Date</h4>')
    grid[0,2] = ipywidgets.HTML('<h4>Polarisation</h4>')
    grid[0,3] = ipywidgets.HTML('<h4>Size</h4>')
    for i in range(len(products_df.index)):
        grid[i+1,0] = ipywidgets.Label(products_df['index'][i])
        grid[i+1,1] = ipywidgets.Label(str(products_df['ingestiondate'][i]))
        grid[i+1,2] = ipywidgets.Label(products_df['polarisationmode'][i])
        grid[i+1,3] = ipywidgets.Label(products_df['size'][i])
        grid[i+1,4] = ipywidgets.Button(description = 'Download')
        grid[i+1,4].on_click(functools.partial(on_downloadButton_clicked, tile_id=products_df.values[i][-1]))
    display(grid)
    
# muestra el índice del producto y la fecha de ingestión en el elemento HTML al pasar el cursor sobre el mosaico de Sentinel-1
def hover_handler(feature, **kwargs):
    # restablecer el widget HTML y establecer nuevos valores
    html.value = '''
    Index: {}<br/>
    <small>Ingestion Date: {}</small>
    '''.format(feature['properties']['index'], feature['properties']['ingestiondate'])

# buscar mosaicos de Sentinel-1 disponibles con respecto a AOI
def on_searchButton_clicked(b):
    # reenfoque del mapa para cubrir mosaicos de Sentinel-1
    download_map.zoom = 6.5
    # crear el elemento WKT con coordenadas geográficas del AOI
    coordinates = draw_control.last_draw['geometry']['coordinates'][0]
    lng1, lat1 = coordinates[0][0], coordinates[0][1]
    lng2, lat2 = coordinates[1][0], coordinates[1][1]
    lng3, lat3 = coordinates[2][0], coordinates[2][1]
    lng4, lat4 = coordinates[3][0], coordinates[3][1]
    footprint = 'POLYGON((%s %s, %s %s, %s %s, %s %s, %s %s))' % (lng1, lat1, lng2, lat2, lng3, lat3, lng4, lat4, lng1, lat1)
    # función de llamada para buscar mosaicos de Sentinel-1 disponibles
    queri(footprint)
    
# descargar el mosaico elegido de Sentinel-1 en la subcarpeta 'entrada'
def on_downloadButton_clicked(b, tile_id):
    # obtener información del producto
    product_info = api.get_product_odata(tile_id)
    # comprobar si el producto está disponible
    if product_info['Online']:
        # compruebe si existe la carpeta de entrada, si no, cree la carpeta de entrada
        input_path = os.path.join(directory, 'input')
        if not os.path.isdir(input_path):
            os.mkdir(input_path)
        # cambiar a la subcarpeta de 'entrada' para almacenar el producto
        os.chdir(input_path)
        # actualizar estado
        print('\nProduct %s is online. Starting download.' % tile_id, flush=True)
        # descargar producto
        api.download(tile_id)
        # volver al directorio de trabajo anterior
        os.chdir(directory)
    # mensaje de error cuando el producto no está disponible
    else:
        print('\nProduct %s is not online. Must be requested manually.\n' % tile_id, flush=True)




####################################################
##################### CÓDIGO ######################
####################################################

# comprobar la entrada del usuario si se solicita la descarga de la imagen
if download['imageDownload']:
    # conectar a la API
    api = SentinelAPI(download['username'], download['password'], 'https://scihub.copernicus.eu/dhus')
    # crear un mapa con funcionalidad de búsqueda
    download_map = ipyleaflet.Map(zoom = 1.4)
    download_map.add_control(ipyleaflet.SearchControl(
        position = 'topleft',
        url = 'https://nominatim.openstreetmap.org/search?format=json&q={s}',
        zoom = 5))
    download_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
    display(download_map)
    
    # crear un widget HTML para mostrar el índice del producto y la fecha de ingestión del mosaico Sentinel-1 seleccionado
    html = ipywidgets.HTML('')
    control = ipyleaflet.WidgetControl(widget=html, position='topright')
    download_map.add_control(control)
    
    # verifique si se proporciona el archivo AOI y conviértalo a JSON para mostrar la AOI en el mapa
    try:
        # llama a la función readJSONfromAOI para obtener GeoJSON de un archivo JSON, SHP o KMZ
        data_json = readJSONFromAOI('%s/AOI' % directory)
        # mostrar la AOI en el mapa de acuerdo con los datos JSON
        aoi = ipyleaflet.GeoJSON(data = data_json, style = {'color' : 'green'})
        download_map.zoom = 6.5
        try:
            # Formato GeoJSON si se proporciona KMZ
            download_map.center = (aoi.data['features'][0]['geometry']['coordinates'][0][0][0][1],
                                   aoi.data['features'][0]['geometry']['coordinates'][0][0][0][0])
        except:
            # Formato GeoJSON si se proporciona JSON o SHP
            download_map.center = (aoi.data['features'][0]['geometry']['coordinates'][0][0][1],
                                   aoi.data['features'][0]['geometry']['coordinates'][0][0][0])
        download_map.add_layer(aoi)
        # buscar mosaicos de Sentinel-1 disponibles según los datos JSON
        footprint = geojson_to_wkt(data_json)
        queri(footprint)

    # si no se proporciona un AOI, debe seleccionarse manualmente
    except FileNotFoundError:
        # crear una función de control de dibujo
        draw_control = ipyleaflet.DrawControl(circlemarker={},
                                              polyline={},
                                              polygon={'shapeOptions':{'color':'green'}},
                                              rectangle={'shapeOptions':{'color':'green'}})
        # agrega la función de control de dibujo al mapa
        download_map.add_control(draw_control)
        # Crear un botón de búsqueda y función de llamada para buscar mosaicos de Sentinel-1 disponibles cuando se hace clic
        searchButton = ipywidgets.Button(description = 'Search')
        searchButton.on_click(on_searchButton_clicked)
        display(searchButton)

## Procesamiento

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart3.png?raw=1" width="1000"/>

Si existe más de una imagen de Sentinel-1 en la subcarpeta *'entrada'*, el usuario puede seleccionar cuál se utilizará para el procesamiento. El subconjunto se genera de acuerdo con el archivo AOI en la subcarpeta *'AOI'*. Si no se proporciona un archivo AOI, un mapa interactivo permite dibujar y descargar el área de interés haciendo clic en el botón *'Exportar'* - dentro del mapa o cargar directamente un archivo AOI almacenado localmente. Posteriormente, se realizan los siguientes pasos de procesamiento:

1. ***Aplicar el archivo de órbita***: El archivo de órbita proporciona información precisa sobre la posición y la velocidad del satélite. Con base en esta información, se actualizan los vectores de estado de la órbita en los metadatos del producto. Los archivos de órbita precisos están disponibles de días a semanas después de la generación del producto. Dado que este es un paso de procesamiento opcional, la herramienta continuará el flujo de trabajo en caso de que el archivo de órbita aún no esté disponible para permitir aplicaciones de mapeo rápido.


2. ***Eliminación de ruido térmico***: la corrección de ruido térmico se aplica a los productos Sentinel-1 Nivel-1 GRD que aún no se han corregido.


3. ***Calibración radiométrica***: El objetivo de esta calibración de SAR es proporcionar imágenes en las que los valores de los píxeles se puedan relacionar directamente con la retrodispersión de la escena de radar. Aunque las imágenes de SAR no calibradas son suficientes para un uso cualitativo, las imágenes de SAR calibradas son esenciales para el uso cuantitativo de los datos de SAR.


4. ***Filtrado de puntos***: las imágenes SAR tienen texturas inherentes llamadas puntos que degradan la calidad de la imagen y dificultan la interpretación de las características. Estos puntos son causados por la interferencia aleatoria constructiva y destructiva de las ondas de retorno desfasadas pero coherentes y retro-dispersadas por la dispersión elemental dentro de cada celda de resolución. La reducción de ruido de estos puntos se puede aplicar mediante filtrado espacial o procesamiento multilook. En este paso se utiliza un filtro de tipo Lee con un tamaño de ventana de 5 x 5.


5. ***Corrección del terreno***: Debido a las variaciones topográficas de una escena y la inclinación del sensor de satélite, las distancias en las imágenes SAR pueden distorsionarse. Los datos que no se dirijan directamente a la ubicación del nadir del sensor tendrán cierta distorsión. Por lo tanto, las correcciones del terreno están destinadas a compensar estas deformaciones para permitir una representación geométrica realista en la imagen.


6. ***Binarización***: para obtener una máscara de inundación binaria, se analiza el histograma para separar el agua de los píxeles que no son de agua. Debido a la geometría lateral de los sensores SAR y la superficie del agua comparativamente lisa, solo una proporción muy pequeña de retrodispersión se refleja en el sensor, lo que genera valores de píxeles comparativamente bajos en el histograma. El umbral utilizado para la separación se calcula automáticamente utilizando las implementaciones de <a href="https://scikit-image.org/"> scikit-image </a> y un uso combinado del <a href = "https: // doi .org / 10.1111 / j.1749-6632.1965.tb11715.x "> método mínimo </a> y el <a href =" https://www.semanticscholar.org/paper/A-threshold-selection-method-from- gray-level-Otsu / 1d4816c612e38dac86f2149af667a5581686cdef? p2df "> método de Otsu </a>. La capa <a href="http://due.esrin.esa.int/page_globcover.php"> GlobCover </a> de la Agencia Espacial Europea se utiliza para enmascarar los cuerpos de agua permanentes.


7. ***Filtrado de manchas***: En este paso se utiliza un filtro de mediana con un tamaño de ventana de 7 x 7.

In [ ]:
# Click para iniciar

####################################################
############ DEFINICIONES DE FUNCIONES #############
####################################################

# crear producto S1
def getScene(path):
    # establecer la ruta correcta del archivo de entrada y crear el producto S1
    if len(files) is 1:
        file_path = path
    else:
        file_path = path.selected
    S1_source = snappy.ProductIO.readProduct(file_path)

    # leer las coordenadas geográficas de los metadatos de imágenes de Sentinel-1
    meta_data = S1_source.getMetadataRoot().getElement('Abstracted_Metadata')
    # ubicar el centro del mapa de acuerdo con la imagen Sentinel-1
    center = (meta_data.getAttributeDouble('centre_lat'), meta_data.getAttributeDouble('centre_lon'))
    locations = [[{'lat' : meta_data.getAttributeDouble('first_near_lat'), 'lng' : meta_data.getAttributeDouble('first_near_long')},
                  {'lat' : meta_data.getAttributeDouble('last_near_lat'),  'lng' : meta_data.getAttributeDouble('last_near_long')},
                  {'lat' : meta_data.getAttributeDouble('last_far_lat'),   'lng' : meta_data.getAttributeDouble('last_far_long')},
                  {'lat' : meta_data.getAttributeDouble('first_far_lat'),  'lng' : meta_data.getAttributeDouble('first_far_long')}]]

    # crea un mapa interactivo
    basic_map = ipyleaflet.Map(center = center, zoom = 7.5)
    # define un polígono fijo que ilustra la imagen de Sentinel-1
    polygon_fix = ipyleaflet.Polygon(locations = locations, color='royalblue')
    basic_map.add_layer(polygon_fix)
    # muestra el mapa
    basic_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
    display(basic_map)
    
    # verifique si se proporciona el archivo del AOI y conviértalo a JSON para mostrar el AOI en el mapa
    try:
        # llama a la función readJSONfromAOI para obtener GeoJSON de un archivo JSON, SHP o KMZ
        data_json = readJSONFromAOI('%s/AOI' % directory)
        # mostrar el AOI en el mapa de acuerdo con los datos JSON
        basic_map.add_layer(ipyleaflet.GeoJSON(data = data_json, style = {'color' : 'green'}))
        # aplicar el subconjunto de acuerdo con los datos JSON
        footprint = geojson_to_wkt(data_json)
        # ejecutar el procesamiento
        processing(S1_source, footprint)
        
    # si no se proporciona un AOI, debe seleccionarse manualmente
    except:
        # polígono editable que determina el AOI
        polygon_flex = ipyleaflet.Polygon(locations = locations,
                                          color = 'green',
                                          fill_color = 'green',
                                          transform = True)
        basic_map.add_layer(polygon_flex)
        # Crear un botón de proceso y la función de llamada para comenzar a procesar cuando se hace clic
        processButton = ipywidgets.Button(description = 'Start Processing')
        processButton.on_click(functools.partial(on_processButton_clicked,
                                                 S1_source = S1_source,
                                                 polygon_flex = polygon_flex))
        display(processButton)
    
# calcular y devolver el umbral de entrada de la 'Banda'
# SNAP API: https://step.esa.int/docs/v6.0/apidoc/engine/
def getThreshold(S1_band):
    # leer la banda
    w = S1_band.getRasterWidth()
    h = S1_band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    S1_band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h * w

    # calcular el umbral usando el método Otsu 
    threshold_otsu = skimage.filters.threshold_otsu(band_data)
    # calcular el umbral utilizando el método mínimo
    threshold_minimum = skimage.filters.threshold_minimum(band_data)
    # obtener el número de píxeles para ambos umbrales
    numPixOtsu = len(band_data[abs(band_data - threshold_otsu) < 0.1])
    numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # si el número de píxeles en el umbral mínimo es inferior al 1% del número de píxeles en el umbral Otsu
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        # ajustar los datos de la banda de acuerdo
        if threshold_otsu < threshold_minimum:
            band_data = band_data[band_data < threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        else:
            band_data = band_data[band_data > threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
    
        numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # seleccionar el umbral final
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        threshold = threshold_otsu
    else:
        threshold = threshold_minimum

    return threshold

# calcular la máscara binaria de la entrada de tipo 'Producto' con respecto a la expresión en la matriz de cadenas
def binarization(S1_product, expressions):

    BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
    targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(expressions))

    # bucle a través de bandas
    for i in range(len(expressions)):
        targetBand = BandDescriptor()
        targetBand.name = '%s' % S1_product.getBandNames()[i]
        targetBand.type = 'float32'
        targetBand.expression = expressions[i]
        targetBands[i] = targetBand
    
    parameters = snappy.HashMap()
    parameters.put('targetBands', targetBands)    
    mask = snappy.GPF.createProduct('BandMaths', parameters, S1_product)

    return mask

# botón de inicio de procesamiento 
def on_processButton_clicked(b, S1_source, polygon_flex):
    # get coordinates and store in WKT format variable
    lng1, lat1 = polygon_flex.locations[0][0]['lng'], polygon_flex.locations[0][0]['lat']
    lng2, lat2 = polygon_flex.locations[0][1]['lng'], polygon_flex.locations[0][1]['lat']
    lng3, lat3 = polygon_flex.locations[0][2]['lng'], polygon_flex.locations[0][2]['lat']
    lng4, lat4 = polygon_flex.locations[0][3]['lng'], polygon_flex.locations[0][3]['lat']
    footprint = 'POLYGON((%s %s, %s %s, %s %s, %s %s, %s %s))' % (lng1, lat1, lng2, lat2, lng3, lat3, lng4, lat4, lng1, lat1)
    # ejecutar el procesamiento
    processing(S1_source, footprint)

# pasos del procesamiento
def processing(S1_source, footprint):
    
    # Operador de recorte o subconjunto
    parameters = snappy.HashMap()
    parameters.put('copyMetadata', True)
    geom = snappy.WKTReader().read(footprint)
    parameters.put('geoRegion', geom)
    parameters.put('sourceBands', sourceBands)
    S1_crop = snappy.GPF.createProduct('Subset', parameters, S1_source)
    # actualizar estado
    print('\nSubset successfully generated.\n', flush=True)
    
    # aplicar el Apply-Orbit-File
    print('1. Apply Orbit File:          ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    # Continuar con el cálculo en caso de que aún no haya ningún archivo de órbita disponible.
    parameters.put('continueOnFail', True)
    S1_Orb = snappy.GPF.createProduct('Apply-Orbit-File', parameters, S1_crop)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Operador de eliminación de ruido térmico
    print('2. Thermal Noise Removal:     ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('removeThermalNoise', True)
    S1_Thm = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, S1_Orb)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # operador de calibración
    print('3. Radiometric Calibration:   ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('outputSigmaBand', True)
    S1_Cal = snappy.GPF.createProduct('Calibration', parameters, S1_Thm)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # operador de filtrado de ruido o Speckle
    print('4. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    S1_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_Cal)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Conversión lineal a db
    S1_Spk_db = snappy.GPF.createProduct('LinearToFromdB', snappy.HashMap(), S1_Spk)

    # Operador de corrección de terreno
    print('5. Terrain Correction:        ', end='', flush=True)
    parameters = snappy.HashMap()
    parameters.put('demName', 'SRTM 1Sec HGT')
    parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('pixelSpacingInMeter', 10.0)
    parameters.put('nodataValueAtSea', False)
    parameters.put('saveSelectedSourceBand', True)
    S1_TC = snappy.GPF.createProduct('Terrain-Correction', parameters, S1_Spk_db)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Binarización
    print('6. Binarization:              ', end='', flush=True)
    start_time = time.time()
    # añadir la banda GlobCover
    parameters = snappy.HashMap()
    parameters.put('landCoverNames', 'GlobCover')
    GlobCover = snappy.GPF.createProduct('AddLandCover', parameters, S1_TC)
    # matriz de cadena vacía para expresiones matemáticas de la banda de binarización
    expressions = ['' for i in range(S1_TC.getNumBands())]
    # matriz vacía para el umbral (s)
    thresholds = np.zeros(S1_TC.getNumBands())
    # bucle a través de bandas
    for i in range(S1_TC.getNumBands()):
        # calcular el umbral de la banda y almacenarlo en una matriz flotante
        # utilice el producto S1_Spk_db por motivos de rendimiento. S1_TC provoca valores 0
        # que distorsionan el histograma y, por lo tanto, el resultado del umbral
        thresholds[i] = getThreshold(S1_Spk_db.getBandAt(i))
        # formular la expresión según el umbral y almacenarla en una matriz de cadenas
        expressions[i] = 'if (%s < %s && land_cover_GlobCover != 210) then 1 else NaN' % (S1_TC.getBandNames()[i], thresholds[i])
    # hacer binarización
    S1_floodMask = binarization(GlobCover, expressions)
    print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

    # Operador de filtro de manchas
    print('7. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Median')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    # definir la máscara de inundación como global para un acceso posterior
    global S1_floodMask_Spk
    S1_floodMask_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_floodMask)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # salida
    if plotResoluts:
        print('8. Plot:                      ', end='', flush=True)
        start_time = time.time()
        for i in range(S1_TC.getNumBands()):
            plotBand(S1_TC.getBandAt(i), thresholds[i])
        print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)




####################################################
###################### CÓDIGO ######################
####################################################

# filtrar la(s) polarización(es) requerida(s) y establecer el nombre del archivo de salida 
if polarisations == 'ambas':
    sourceBands = 'Amplitude_VH,Intensity_VH,Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VHVV'
elif polarisations == 'VH':
    sourceBands = 'Amplitude_VH,Intensity_VH'
    output_extensions   = 'processed_VH'
elif polarisations == 'VV':
    sourceBands = 'Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VV'

# ruta del archivo de entrada .zip de Sentinel-1
input_path = os.path.join(directory, 'input')
# matriz de cadena vacía para almacenar archivos Sentinel-1 en la subcarpeta 'entrada'
files = []
# agregar archivos a la lista
for file in glob.glob1(input_path, '*.zip'):
    files.append(file)
# seleccione el archivo de entrada y comience a procesar si solo hay un archivo Sentinel-1 disponible
if len(files) == 1:
    input_name = files[0]
    print('Selected:  %s\n' % input_name, flush=True)
    # aplicar subconjunto de acuerdo con los datos JSON
    getScene('%s/%s' % (input_path, input_name))
# abrir diálogo para seleccionar el archivo de entrada si hay más de uno disponible
else:
    print('More or less than one Sentinel-1 files have been found. Please select.', flush=True)
    fc = FileChooser(input_path)
    fc.register_callback(getScene)
    display(fc)

## Exportar Datos

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart4.png?raw=1" width="1000"/>

La máscara de inundación procesada se exporta como un GeoTIFF, SHP, KML y GeoJSON y se almacena en la subcarpeta *'salida'*. Un mapa interactivo muestra la máscara de inundación.

In [ ]:
# Click para iniciar

####################################################
###################### CÓDIGO ######################
####################################################

print('Exporting...\n', flush=True)
# compruebe si existen carpetas de salida, si no crea las carpetas
output_path = os.path.join(directory, 'output')
if not os.path.isdir(output_path):
    os.mkdir(output_path)
GeoTIFF_path = os.path.join(output_path, 'GeoTIFF')
if not os.path.isdir(GeoTIFF_path):
    os.mkdir(GeoTIFF_path)
SHP_path = os.path.join(output_path, 'SHP')
if not os.path.isdir(SHP_path):
    os.mkdir(SHP_path)
KML_path = os.path.join(output_path, 'KML')
if not os.path.isdir(KML_path):
    os.mkdir(KML_path)
GeoJSON_path = os.path.join(output_path, 'GeoJSON')
if not os.path.isdir(GeoJSON_path):
    os.mkdir(GeoJSON_path)
# obtener el nombre del archivo si se utilizó el selector de archivos
if len(files) is not 1: input_name = fc.selected_filename

# escribir el archivo de salida como GeoTIFF
print('1. GeoTIFF:                   ', end='', flush=True)
start_time = time.time()
snappy.ProductIO.writeProduct(S1_floodMask_Spk, '%s/%s_%s' % (GeoTIFF_path, os.path.splitext(input_name)[0], output_extensions), 'GeoTIFF')
print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

# convertir el GeoTIFF a SHP
print('2. SHP:                       ', end='', flush=True)
start_time = time.time()
# permitir que GDAL lance excepciones de Python
gdal.UseExceptions()
open_image = gdal.Open('%s/%s_%s.tif' % (GeoTIFF_path, os.path.splitext(input_name)[0], output_extensions))
srs = osr.SpatialReference()
srs.ImportFromWkt(open_image.GetProjectionRef())
shp_driver = ogr.GetDriverByName('ESRI Shapefile')
# matriz de cadena vacía para bandas en GeoTIFF
output_shp = ['' for i in range(open_image.RasterCount)]
if open_image.RasterCount == 1:
    output_shp[0] = '%s/%s_processed_%s' % (SHP_path, os.path.splitext(input_name)[0], polarisations)
else:
    VH_SHP_path = os.path.join(SHP_path, 'VH')
    if not os.path.isdir(VH_SHP_path):
        os.mkdir(VH_SHP_path)
    VV_SHP_path = os.path.join(SHP_path, 'VV')
    if not os.path.isdir(VV_SHP_path):
        os.mkdir(VV_SHP_path)
    output_shp[0] = '%s/%s_processed_VH' % (VH_SHP_path, os.path.splitext(input_name)[0])
    output_shp[1] = '%s/%s_processed_VV' % (VV_SHP_path, os.path.splitext(input_name)[0])
# bucles a través de bandas en GeoTIFF
for i in range(open_image.RasterCount):
    input_band = open_image.GetRasterBand(i+1)
    output_shapefile = shp_driver.CreateDataSource(output_shp[i] + '.shp')
    new_shapefile = output_shapefile.CreateLayer(output_shp[i], srs=srs)
    new_shapefile.CreateField(ogr.FieldDefn('DN', ogr.OFTInteger))
    gdal.Polygonize(input_band, input_band.GetMaskBand(), new_shapefile, 0, [], callback=None)
    # filtra atributos con valores distintos de 1 (debe ser NaN o el valor respectivo)
    new_shapefile.SetAttributeFilter('DN != 1')
    for feat in new_shapefile:
        new_shapefile.DeleteFeature(feat.GetFID())
    new_shapefile.SyncToDisk()
print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

# convertir de SHP a KML
print('3. KML:                       ', end='', flush=True)
start_time = time.time()
if open_image.RasterCount == 1:
    shp_file = gdal.OpenEx('%s/%s_processed_%s.shp' % (SHP_path, os.path.splitext(input_name)[0], polarisations))
    ds = gdal.VectorTranslate('%s/%s_processed_%s.kml' % (KML_path, os.path.splitext(input_name)[0], polarisations), shp_file, format='KML')
    del ds
else:
    shp_file_VH = gdal.OpenEx('%s/%s_processed_VH.shp' % (VH_SHP_path, os.path.splitext(input_name)[0]))
    ds_VH = gdal.VectorTranslate('%s/%s_processed_VH.kml' % (KML_path, os.path.splitext(input_name)[0]), shp_file_VH, format='KML')
    del ds_VH
    shp_file_VV = gdal.OpenEx('%s/%s_processed_VV.shp' % (VV_SHP_path, os.path.splitext(input_name)[0]))
    ds_VV = gdal.VectorTranslate('%s/%s_processed_VV.kml' % (KML_path, os.path.splitext(input_name)[0]), shp_file_VV, format='KML')
    del ds_VV
print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

# convertir de SHP a GeoJSON
print('4. GeoJSON:                   ', end='', flush=True)
start_time = time.time()
if open_image.RasterCount == 1:
    shp_file = geopandas.read_file('%s/%s_processed_%s.shp' % (SHP_path, os.path.splitext(input_name)[0], polarisations))
    shp_file.to_file('%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], polarisations), driver='GeoJSON')
else:
    shp_file_VH = geopandas.read_file('%s/%s_processed_VH.shp' % (VH_SHP_path, os.path.splitext(input_name)[0]))
    shp_file_VH.to_file('%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')    
    shp_file_VV = geopandas.read_file('%s/%s_processed_VV.shp' % (VV_SHP_path, os.path.splitext(input_name)[0]))
    shp_file_VV.to_file('%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')
print('--- %.2f seconds ---\n' % (time.time() - start_time), flush=True)
print('Files successfuly stored under %s.\n' % output_path, flush=True)

# gráficar resultados
results_map = ipyleaflet.Map(zoom=9, basemap=ipyleaflet.basemaps.OpenStreetMap.Mapnik)    
if open_image.RasterCount == 1:
    file = '%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], polarisations)
    with open(file, 'r') as f:
        data_json = json.load(f) 
    mask = ipyleaflet.GeoJSON(data = data_json, name = 'Flood Mask', style = {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'})
    results_map.add_layer(mask)
    results_map.center = (mask.data['features'][0]['geometry']['coordinates'][0][0][1],
                          mask.data['features'][0]['geometry']['coordinates'][0][0][0])
else:
    file_VV = '%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
    with open(file_VV, 'r') as f_VV:
        data_json_VV = json.load(f_VV)
    mask_VV = ipyleaflet.GeoJSON(data = data_json_VV, name = 'Flood Mask: VV', style = {'color':'red', 'opacity':'1', 'fillColor':'red', 'fillOpacity':'1', 'weight':'0.8'})
    results_map.add_layer(mask_VV)
    results_map.center = (mask_VV.data['features'][0]['geometry']['coordinates'][0][0][1],
                          mask_VV.data['features'][0]['geometry']['coordinates'][0][0][0])  
    file_VH = '%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
    with open(file_VH, 'r') as f_VH:
        data_json_VH = json.load(f_VH)
    mask_VH = ipyleaflet.GeoJSON(data = data_json_VH, name = 'Flood Mask: VH', style = {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'})
    results_map.add_layer(mask_VH)
results_map.add_control(ipyleaflet.FullScreenControl())
results_map.add_control(ipyleaflet.LayersControl(position='topright'))
results_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
display(results_map)